In [61]:
import mne
import os
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm

In [11]:
def dump_data(data, filename):
    print('writing file: ' + filename)
    with open(filename, 'wb') as f:
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

### Pickle files 

In [65]:
sub = 5
cond = "img" #"fix"
highpass = None
lowpass = 40
trialwin = [-.2, .7]

# Paths
vhdr_file = f'/projects/crunchie/boyanova/EEG_Things/eeg_experiment/eeg_raw/sub{sub:02d}/eeg_things_{cond}_{sub:04d}.vhdr'
beh_file = pd.read_csv(f'/projects/crunchie/boyanova/EEG_Things/eeg_experiment/beh/sub{sub:02d}/{sub}_eeg_exp_{cond}.csv')

# Load the raw data
raw = mne.io.read_raw_brainvision(vhdr_file, preload=True)

# Filter
eeg_picks = mne.pick_types(raw.info, eeg=True)
raw = raw.copy().filter(l_freq=highpass,
                        h_freq=lowpass,
                        picks=eeg_picks)

# Get events from eeg
events, events_id = mne.events_from_annotations(raw)
events  = events[2:, :]
allowed_events = np.array([1, 2, 3, 4, 11, 12, 13, 14, 201, 202, 203, 204, 211, 212, 213, 214]) 
events_filtered = events[np.isin(events[:, 2], allowed_events)]


# Get epochs
epochs = mne.Epochs(raw, events_filtered, tmin=trialwin[0],
                    tmax=trialwin[1], picks='eeg',
                    baseline=(None, 0), preload=True,
                    reject=None)


# Get data: I need to calculate which trials to remove
dat = {"eeg": epochs.get_data(),
       "time": epochs.times,
       "ids": events_filtered[:, 2],
       "channels": epochs.ch_names,
       "button_press_mask": np.isin(beh_file['key_resp_3.keys'].values[0:-1], 'space'),
       "block_type": beh_file['block_type'].values[0:-1] ,
       "block_num": np.array([int(x.split("_")[-1].split(".")[0]) for x in beh_file['block_name'].values[0:-1]])}

dat_name = "/projects/crunchie/boyanova/EEG_Things/eeg_experiment/eeg_epoched/eeg_things_{:04d}_{}.pickle".format(sub, cond)
dump_data(dat, dat_name)



Extracting parameters from /projects/crunchie/boyanova/EEG_Things/eeg_experiment/eeg_raw/sub05/eeg_things_img_0005.vhdr...
Setting channel info structure...
Reading 0 ... 1809979  =      0.000 ...  1809.979 secs...
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 331 samples (0.331 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s


Used Annotations descriptions: ['Comment/ControlBox is not connected via USB', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S 11', 'Stimulus/S 12', 'Stimulus/S 13', 'Stimulus/S 14', 'Stimulus/S 15', 'Stimulus/S201', 'Stimulus/S202', 'Stimulus/S203', 'Stimulus/S204', 'Stimulus/S211', 'Stimulus/S212', 'Stimulus/S213', 'Stimulus/S214']
Not setting metadata
1440 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1440 events and 901 original time points ...


[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    2.7s finished


0 bad epochs dropped
writing file: /projects/crunchie/boyanova/EEG_Things/eeg_experiment/eeg_epoched/eeg_things_0005_img.pickle


### Checking extra events

In [49]:
# Get unique event codes from EEG
eeg_event_codes = set(events[:, 2])

# Get unique event codes from behavior
beh_event_codes = set(beh_file['trig'][0:-1])  # Replace with actual column name

# Find the extra event code
extra_events = eeg_event_codes - beh_event_codes
print(f"Extra events: {extra_events}")

# If there are extra events, find their indices
if extra_events:
    extra_event_code = list(extra_events)[0]  # Assuming one extra event
    extra_event_indices = [i for i, event in enumerate(events) if event[2] == extra_event_code]
    print(f"Indices of the extra event ({extra_event_code}): {extra_event_indices}")
else:
    print("No extra events found.")


Extra events: {15}
Indices of the extra event (15): [0]


### Renaming eeg files

In [4]:
from mne_bids.copyfiles import copyfile_brainvision
import mne
# Rename the file
sub = 0
vhdr_file_path_orig = '/projects/crunchie/boyanova/EEG_Things/eeg_experiment/eeg_raw/sub0_fix/eeg_things_{}_{:04d}.vhdr'.format("img", sub)

vhdr_file_renamed ='/projects/crunchie/boyanova/EEG_Things/eeg_experiment/eeg_raw/sub0/eeg_things_{}_{:04d}.vhdr'.format("fix", sub)
copyfile_brainvision(vhdr_file_path_orig, vhdr_file_renamed, verbose=True)

# Check that MNE-Python can read in both, the original as well as the renamed
# data (two files: their contents are the same apart from the name)
raw = mne.io.read_raw_brainvision(vhdr_file_path_orig)
raw_renamed = mne.io.read_raw_brainvision(vhdr_file_renamed)

Created "eeg_things_fix_0000.eeg" in "/projects/crunchie/boyanova/EEG_Things/eeg_experiment/eeg_raw/sub0".
Created "eeg_things_fix_0000.vhdr" in "/projects/crunchie/boyanova/EEG_Things/eeg_experiment/eeg_raw/sub0".
Created "eeg_things_fix_0000.vmrk" in "/projects/crunchie/boyanova/EEG_Things/eeg_experiment/eeg_raw/sub0".
Extracting parameters from /projects/crunchie/boyanova/EEG_Things/eeg_experiment/eeg_raw/sub0_fix/eeg_things_img_0000.vhdr...
Setting channel info structure...
Extracting parameters from /projects/crunchie/boyanova/EEG_Things/eeg_experiment/eeg_raw/sub0/eeg_things_fix_0000.vhdr...
Setting channel info structure...


### Concatenating pickles

In [36]:
import os
import pickle
import numpy as np

path = "/projects/crunchie/boyanova/EEG_Things/eeg_experiment/eeg_epoched/fix_combine/"
files = os.listdir(path)

def load_data(file):
    print('loading file: ' + file)
    with open(file, 'rb') as f:
        data = pickle.load(f)
    return data

# Updated helper function to concatenate two dictionaries with np.arrays,
# keeping "channels" and "time" from the first dictionary only
def concatenate_dicts(dict1, dict2):
    combined_dict = {}
    for key in dict1:
        if key in ["channels", "time"]:
            combined_dict[key] = dict1[key]
        else:
            combined_dict[key] = np.concatenate((dict1[key], dict2[key]), axis=0)
    return combined_dict


concatenated_data = {}
for file in files:
    # Check if file ends in '_fix.pickle' and has a corresponding '_fix_2.pickle'
    if file.endswith('_fix.pickle'):
        file_2 = file.replace('_fix.pickle', '_fix_2.pickle')
        
        if file_2 in files:
            print(f"Concatenating {file} and {file_2}")
            
            # Load both dictionaries
            data1 = load_data(os.path.join(path, file))
            data2 = load_data(os.path.join(path, file_2))
            
            combined_data = concatenate_dicts(data1, data2)
            concatenated_data = combined_data

dat_name = "/projects/crunchie/boyanova/EEG_Things/eeg_experiment/eeg_epoched/eeg_things_{:04d}_{}.pickle".format(sub, cond)
dump_data(concatenated_data, dat_name)


Concatenating eeg_things_0001_fix.pickle and eeg_things_0001_fix_2.pickle
loading file: /projects/crunchie/boyanova/EEG_Things/eeg_experiment/eeg_epoched/fix_combine/eeg_things_0001_fix.pickle
loading file: /projects/crunchie/boyanova/EEG_Things/eeg_experiment/eeg_epoched/fix_combine/eeg_things_0001_fix_2.pickle
writing file: /projects/crunchie/boyanova/EEG_Things/eeg_experiment/eeg_epoched/eeg_things_0001_fix.pickle


In [37]:
concatenated_data['button_press_mask'].shape

(2880,)